In [1]:
import pandas as pd

In [2]:
# df=pd.read_csv('generated_dataset.csv')
df=pd.read_csv('input_output.csv')

In [3]:
df.head(1)

,input_text,target_text
0,Excursion details -1st Stage Separator train B...,If feasible line up 34' Bypass in line whene...


In [4]:
input_texts=df['input_text'].to_list()
target_texts=df['target_text'].to_list()

In [5]:
target_texts = ['\t ' + text + ' \n' for text in target_texts]

In [6]:
# # Sample dataset
# input_texts = ['I am a student.', 'This is a pen.', 'You have a book.']
# target_texts = ['Je suis étudiant.', 'Ceci est un stylo.', 'Tu as un livre.']

# # Add start and end tokens to target texts
# target_texts = ['\t ' + text + ' \n' for text in target_texts]


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize input texts
input_tokenizer = Tokenizer(char_level=False)
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_sequences = pad_sequences(input_sequences, padding='post')

# Tokenize target texts
target_tokenizer = Tokenizer(char_level=False, filters='')
target_tokenizer.fit_on_texts(target_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)
target_sequences = pad_sequences(target_sequences, padding='post')

# Vocabulary sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

# Maximum sequence lengths
max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in target_sequences])


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(128, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & summarize
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, None, 256)         │          48,384 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, None, 256)         │          27,648 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 128), (None,      │         197,120 │ embedding[0][0]            │
│                               │ 128), (None, 128)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, None, 128),       │         197,120 │ embedding_1[0][0],         │
│                               │ (None, 128), (None, 128)] │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 108)         │          13,932 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 484,204 (1.85 MB)

 Trainable params: 484,204 (1.85 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
import numpy as np

decoder_input_data = target_sequences[:, :-1]
decoder_target_data = target_sequences[:, 1:]
decoder_target_data = np.expand_dims(decoder_target_data, -1)

model.fit(
    [input_sequences, decoder_input_data], decoder_target_data,
    #batch_size=64,
    epochs=100,
    validation_split=0.2
)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 799ms/step - accuracy: 0.1683 - loss: 4.5836 - val_accuracy: 0.3535 - val_loss: 4.0317
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.3482 - loss: 3.6867 - val_accuracy: 0.3352 - val_loss: 3.4543
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.3521 - loss: 3.1901 - val_accuracy: 0.3352 - val_loss: 3.3298
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.3566 - loss: 3.0075 - val_accuracy: 0.3535 - val_loss: 3.2918
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.3507 - loss: 3.0142 - val_accuracy: 0.3535 - val_loss: 3.2963
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.3680 - loss: 2.8345 - val_accuracy: 0.3571 - val_loss: 3.2611
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.3674 - loss: 2.8470 - val_accuracy: 0.3516 - val_loss: 3.3535
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.3743 - loss: 2.7266 - val_accuracy: 0.3352 - 

In [10]:
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [11]:
def decode_sequence(input_text):
    # Tokenize the input text
    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding='post')
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1 with only the start character.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character code.
    # Note: "\t" is used as the "start sequence" character for the targets, and "\n" as the "end sequence" character.
    target_seq[0, 0] = target_tokenizer.word_index['\t']
    
    # Looping variable to keep track of when to stop the loop (when we hit the max length or find the stop character)
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_tokenizer.index_word[sampled_token_index]
        decoded_sentence += sampled_char+' '
        
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence.split()) > max_decoder_seq_length):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
    
    return decoded_sentence


In [28]:
# input_text = "Excursion details -1st Stage Separator train B oil level went more than high high limit 2 to 3 time i.e. more than 89.5% after startup.Level control valve was 100% open  working as per level control philosphy  but more flow can be achieved through"
input_text = "What is teh recommendation for higher pressure system"
decoded_sentence = decode_sequence(input_text)
print("Input:", input_text)
print("Output:", decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Input: What is teh recommendation for higher pressure system
Output: try to do manual backwash for short duration if air scorur can be available partaily. 
 


In [29]:
input_text = "Excursion details -1st Stage Separator train B oil level went more than high high limit 2 to 3 time i.e. more than 89.5% after startup.Level control valve was 100% open  working as per level control philosphy  but more flow can be achieved through"
# input_text = "What is teh recommendation for higher pressure system"
decoded_sentence = decode_sequence(input_text)
print("Input:", input_text)
print("Output:", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Input: Excursion details -1st Stage Separator train B oil level went more than high high limit 2 to 3 time i.e. more than 89.5% after startup.Level control valve was 100% open  working as per level control philosphy  but more flow can be achieved through
Output: check the flare knock out drum level. 
 


In [14]:
# saving the model
# model.save('saved_model.keras')

In [19]:
# loading the model
# import tensorflow as tf
# loaded_model = tf.keras.models.load_model("saved_model.keras")